# SENTIMENT-BASED STOCK MARKET PREDICTION
By Gauri Narayan & Raefah Wahid

In [1]:
import pandas as pd
from demo import split, convert_data, var_model, mse

## Goal
A common technique of stock market prediction is to utilize textual data alongside stock data [1]. Researchers have explored different Natural Language Processing techniques when working with textual information as well as experimented with different prediction models. Our goal for this project is to evaluate the different models that can be used to tackle the problem of stock market prediction.

## Data
For this demo, we will be looking at the company Amazon. Ten tweets were gathered per day for each of 10 selected company over the year 2019 and cleaned. Stock data was collected in the form of closing prices from a public dataset [6].

# Modeling

## Naive Bayes
As a baseline for sentiment analysis, we began by implementing the Naive Bayes model. Though Naive Bayes assumes independence, it is a standard approach to text classification and a good way to test the performance of our forecasting model later on. The Naive Bayes model implemented assigns a positive and negative sentiment score to each tweet using the formula

$$\hat{y} = \frac{p(S_k) \cdot \prod_{i=1}^n p(x_i \mid S_k)}{\prod_{i=1}^n p(x_i)},$$

where $\hat{y}$ is the resulting sentiment score and our latent variable, $S_k$ is one of two sentiment labels (positive or negative), and $x_i$ is a particular word in the tweet. To train this model, we used a public dataset of standard positive and negative tweets that were already labelled. After computing prior probabilities based on this training data, we created a Naive Bayes model that read in tweets related to our chosen companies and computed those tweets’ likelihoods and resulting log sentiment scores. Some days produced no tweets that mentioned the company; for these instances, a sentiment value of 0 was assigned. 

## Vector Autoregression
Vector Autoregression (VAR) is often used for time series dependent forecasting due to the fact it models time series as a linear combination of their past values and the values of other time series. Since our stock information only deals with end-of-day results, we averaged the sentiment scores for tweets across each day for each company. With a lag of 1, our model was formatted as the following linear regression

$$y_t = \theta_1 x_{t- \text{lag}} + \theta_2 y_{t - \text{lag}} + b,$$

where $y_t$ is the closing price of the current day’s stock, $x_{t- \text{lag}}$ is the previous day’s average positive or negative sentiment, $y_{t - \text{lag}}$ is the previous day’s closing price, $\theta_1$ and $\theta_2$ are weights, and $b$ is the bias.

To implement this, we used Pyro’s linear regression module. We use a mean squared error (MSE) loss and optimized using Adam. We experimented with different lags but found that lag = 1 produces the least MSE for both positive and negative sentiment.

In [2]:
data = pd.read_csv('./lag_data/example1.csv')
del data['Unnamed: 0']

In [3]:
train_data, test_data = split(data)

In [4]:
y, pos_x, neg_x = convert_data(train_data)
print("\n Amazon with lag = 1 and positive sentiment: ")
pos_equation = var_model(pos_x, y, 1500)
print("\n Amazon with lag = 1 and negative sentiment: ")
neg_equation = var_model(neg_x, y, 1500)


 Amazon with lag = 1 and positive sentiment: 
Learned parameters:
weight [[0.9954611  0.17504255]]
bias [2.2517302]

 Amazon with lag = 1 and negative sentiment: 
Learned parameters:
weight [[0.99769807 0.17098358]]
bias [0.49300736]


Due to the highly dynamic nature of both sentiment and the stock market, forecasting with a lag of more than seven days is unlikely to be effective. We began with a lag = 1, as seen above, but we experimented with a lag = 3 and 5 as well. 

# Inference
## Granger Causality
An important step is to infer any correlation between sentiment and stock. For this, we will use Granger causality (implemented from a statsmodels library), which can determine whether changes in a time series X can forecast changes in a time series Y [1]. We used a Granger causality test to find the p-values for Amazon's stocks closing prices as influenced by positive and negative sentiment. Neither of these p-values met any significant threshold for correlation, meaning we cannot reject the null hypothesis and determine that there is a correlation between sentiment and stock.

# Evaluation
## Mean Squared Error
To evaluate our model, we utilized MSE as a metric. For each company’s dataset, we split the data into training (80%) and testing (20%) set. We ran VAR on the training set to retrieve weights and biases for each company, and then used this output to predict closing prices of the testing set:

In [5]:
error = mse(test_data, pos_equation, 'pos')
print('The MSE of Amazon is', error, 
      'using positive sentiment as a parameter.')
error = mse(test_data, neg_equation, 'neg')
print('The MSE of Amazon is', error, 
      'using negative sentiment as a parameter.\n')

The MSE of Amazon is 372.8459592217178 using positive sentiment as a parameter.
The MSE of Amazon is 364.0690791376291 using negative sentiment as a parameter.



# Modeling
## Dirichlet Process Mixture Model (DPM)

DPM is a "topic" finding algorithm, which is useful because unlike Naive Bayes, DPM does not assume tweets across multiple days are independent or that they have the same word-sentiment distribution. Given an upper bound of topics in practice, this model can estimate the number of topics, which is extremely useful with real data. Our base layer DPM model assumes that the tweets are trials from a mixture of multinomial distributions. Each word in each tweet is considered to be an independent multinomial trial. We implemented the model in Pyro and used Pyro's Stochastic Variational Inference (SVI) algorithm with hyperparameters K=8, $\alpha=0.1$, and the Adam optimizer to infer parameters. The construction of our standard DPM for day=0 is as follows:

With $k=1,...K$ being the topics, for each observation $x_1, ..., x_N \in \mathbb{R}^C$

$$x_i | z_i, {\theta_i} \sim Mult(\theta_{z_i})$$

$$z_i | \beta \sim Categorical (stickbreak(\beta))$$

$$\theta_i | G \sim G_0$$

$$G_0 | \tau  \sim Dirichlet((\tau_1, ..., \tau_C) = (1/C, ..., 1/C)$$

$$\beta \sim Beta(1, \kappa = \alpha).$$

Our variational parameters will therefore be $\tau$, $\phi$, and $\kappa$. We will sample them from 

$$\phi \sim Dirichlet(1/K, ..., 1/K)$$

$$\tau_k \sim Normal(0.5, 0.25)$$

$$\kappa \sim Unif(0,2).$$

We use Stochastic Variational Inference, which optimizes over the posterior distribution

$$q(\beta, \theta, z) = \Pi_{k=1}^{K−1}q(\beta_k)\Pi_{k=1}^{K}q_k(\theta_k)\Pi_{n=1}^{N}q_n(z).$$


Our construction for the in-sequence DPM  has a similar structure, but with additional variables. Each day's DPM takes as priors for its parameters the optimized values of the previous day. 

For each observation, there are three possibilities: 

1) It is a new topic entirely, wherein the topic's $\theta$ samples take the symmetric Dirichlet distribution as their prior. 

2) It is a new topic linked to an old topic, wherein the topic's $\theta$ samples take as their prior the old topic's estimated Dirichlet distribution.

3) It is an old topic, wherein the other variables are bypassed and the topic's $\theta$ samples are the old topic's estimated $\theta$s. 

There are three added indicator variables in the model: if it is a new topic, if it is linked, and which topic it is linked to. Each of these is sampled from Multinomial distributions. The additional variational parameters are the corresponding probability parameters of these multinomial distributions. 

After assigning each tweet a topic, we computed the mean sentiment score for each topic across the data set, including topics which appear across multiple days. We then ran VAR on the resulting dataset.

In [6]:
# DPM VAR output for Amazon over one month
data = pd.read_csv('./lag_data/amzn_topic_example.csv')
del data['Unnamed: 0']

In [7]:
train_data, test_data = split(data)
y, pos_x, neg_x = convert_data(train_data)
print("\n Amazon with lag = 1 and positive sentiment: ")
pos_equation = var_model(pos_x, y, 1500)
print("\n Amazon with lag = 1 and negative sentiment: ")
neg_equation = var_model(neg_x, y, 1500)


 Amazon with lag = 1 and positive sentiment: 
Learned parameters:
weight [[0.99973583 0.08125562]]
bias [1.2600673]

 Amazon with lag = 1 and negative sentiment: 
Learned parameters:
weight [[0.99978423 0.19138148]]
bias [1.0076685]


In [8]:
error = mse(test_data, pos_equation, 'pos')
print('The MSE of Amazon is', error, 
      'using positive sentiment as a parameter.')
error = mse(test_data, neg_equation, 'neg')
print('The MSE of Amazon is', error, 
      'using negative sentiment as a parameter.\n')

The MSE of Amazon is 327.9451069548925 using positive sentiment as a parameter.
The MSE of Amazon is 328.1148274644477 using negative sentiment as a parameter.



# Conclusion

Granger causality inference we used on both modeling techniques produced p-values that were not less than 0.05, meaning that we cannot reject the null hypothesis. It appears, then, that there is no correlation between sentiment and stock closing prices. However, there is a large decrease in the p-values associated with negative sentiment compared to positive sentiment, suggesting that some correlation exists and could be better captured with more refined models. The Naive Bayes allows us a simple baseline. When we compare DPM to it, we see from the MSE that the improvement seems to be marginal—but because of how computationally heavy DPM is, we were only able to find topic-based sentiment over the month of January. With a larger dataset and further work, such as hyperparameter tuning of the maximum number of topics and the scaling parameters, we could further improve the DPM results that are computed alongside stock data using VAR. This may result in clearer correlation between a particular sentiment and company's stock.

# References

[1] Bollen, J., Mao, H., & Zeng, X. (2011). Twitter mood predicts the stock market. \textit{Journal of Computational Science}, 2(1), 1-8. doi:10.1016/j.jocs.2010.12.007

[2] Si, J., Mukherjee, A., Liu B., Li, Q., Li, H., & Deng, X. (2013). Exploiting Topic based Twitter Sentiment for Stock Prediction. 10.13140/2.1.3604.7043. 

[3] Sun, Y., Gupta, M., Tang, J., Zhao B., Han, J. (2010). Community Evolution Detection in Dynamic Heterogeneous Information Networks. 10.1145/1830252.1830270.

[4] The linear regression used in the VAR model was based on the Pyro module example (https://docs.pyro.ai/en/stable/_modules/pyro/nn/module.html).

[5] The DPM model was based on the stick-breaking formulation in the Pyro documentation example (https://pyro.ai/examples/dirichlet_process_mixture.html).

[6] Porter, Andrew. Yahoo-historical. https://github.com/AndrewRPorter/yahoo-historical